In [1]:
import os
import numpy as np
import pandas as pd
import nltk
from nltk.stem.snowball import SnowballStemmer
from url_sequences.sequence_manager import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
print os.getcwd()

/home/chris/workspace/jupyter-notebook/url2vec


In [3]:
filepath = "/home/chris/Scrivania/url2vec/cs.illinois.eduNoConstraint.words1000.depth.10/vertex.txt"
content_map = get_content_map(filepath)

# english stopwords
stopwords = nltk.corpus.stopwords.words('english')
# stemmer = SnowballStemmer("english")

# map -> {code: token_list}
tokens_map = to_tokens_map(content_map)
# map -> {code: stem_list}
stems_map = to_stems_map(content_map)

# total vocabulary, list of tokens
totalvocab_stemmed = get_total_vocab(stems_map)
# total vocabulary, list of stems
totalvocab_tokenized = get_total_vocab(tokens_map)

# document list
documents = [content_map[key] for key in content_map]

In [ ]:
# to dowload english stopwords
nltk.download()

In [4]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print vocab_frame[:5]

             words
depart  department
of              of
comput    computer
scienc     science
at              at


## TDM
![Alt text](http://www.jiem.org/index.php/jiem/article/viewFile/293/252/2402 "Very nice")

In [5]:
""" 
max_df: 
    this is the maximum frequency within the documents a given feature
    can have to be used in the tfi-idf matrix.
min_idf:
    this could be an integer (e.g. 0.2) and the term would have to be in
    at least 20% of the documents to be considered.
ngram_range:
    (e.g. 1,3) this just means I'll look at unigrams, bigrams and trigrams. 
"""
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.8,
    max_features=200000,
    min_df=0.2,
    stop_words='english',
    use_idf=True,
    tokenizer= tokenize_and_stem,
    ngram_range=(1,3)
)

#fit the vectorizer to synopse
tfidf_matrix = tfidf_vectorizer.fit_transform(documents) 
print(tfidf_matrix.shape)

(728, 156)


In [6]:
terms = tfidf_vectorizer.get_feature_names()
dist = 1 - cosine_similarity(tfidf_matrix)